In [ ]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

# Define paths
project_root = Path().resolve().parent
teams_peak_file = project_root / "data" / "teams_peak_36.csv"
output_file = project_root / "data" / "team_name_to_id.csv"

# Load the teams_peak_36.csv file
df = pd.read_csv(teams_peak_file)

# Normalize team names (lowercase and replace spaces with hyphens)
def normalize_team_name(name):
    """Normalize team name by lowercasing and replacing spaces with hyphens."""
    if pd.isna(name):
        return name
    return str(name).lower().strip().replace(' ', '-')

# Create mapping dataframe with normalized team names
team_name_to_id = pd.DataFrame({
    'team_name': df['name'].apply(normalize_team_name),
    'team_id': df['team_id']
})

# Remove any duplicate team names (if any) - keep first occurrence
team_name_to_id = team_name_to_id.drop_duplicates(subset='team_name', keep='first')

# Sort by team_id for consistency
team_name_to_id = team_name_to_id.sort_values('team_id').reset_index(drop=True)

# Save to CSV
team_name_to_id.to_csv(output_file, index=False)

print(f"✅ Created team name to ID mapping with {len(team_name_to_id)} teams")
print(f"Saved to: {output_file}")
print("\nFirst few rows:")
print(team_name_to_id.head(10))

✅ Created team name to ID mapping with 68 teams
Saved to: /Users/jayesh/University/CS-IV-year/COMPSCI 4AL3/Project Progress Report/emils-demos/data/team_name_to_id.csv

First few rows:
           team_name  team_id
0  ninjas-in-pyjamas     4411
1               mouz     4494
2      natus-vincere     4608
3               pain     4773
4              tyloo     4863
5               ence     4869
6              3dmax     4914
7             fnatic     4991
8         complexity     5005
9           bluejays     5347


In [2]:
# Define paths
project_root = Path().resolve().parent
team_results_dir = project_root / "data" / "team_results"
output_file = project_root / "data" / "map_name_to_id.csv"

# Get the first few CSV files from team_results
team_results_files = sorted(team_results_dir.glob("*.csv"))
num_files_to_process = min(10, len(team_results_files))  # Process first 10 files
files_to_process = team_results_files[:num_files_to_process]

print(f"Processing {num_files_to_process} files to extract map information...")
print(f"Files: {[f.name for f in files_to_process]}\n")

# Collect all map data from the files
all_maps = []

for file_path in files_to_process:
    try:
        df = pd.read_csv(file_path)
        if 'map_name' in df.columns and 'map_id' in df.columns:
            # Get unique map_name and map_id pairs
            maps = df[['map_name', 'map_id']].drop_duplicates()
            all_maps.append(maps)
            print(f"  ✅ {file_path.name}: Found {len(maps)} unique maps")
    except Exception as e:
        print(f"  ❌ Error reading {file_path.name}: {e}")

# Combine all maps and get unique pairs
if all_maps:
    combined_maps = pd.concat(all_maps, ignore_index=True)
    unique_maps = combined_maps.drop_duplicates(subset=['map_name', 'map_id'], keep='first')
    
    # Normalize map names (lowercase and replace spaces with hyphens)
    def normalize_map_name(name):
        """Normalize map name by lowercasing and replacing spaces with hyphens."""
        if pd.isna(name):
            return name
        return str(name).lower().strip().replace(' ', '-')
    
    # Create mapping dataframe with normalized map names
    map_name_to_id = pd.DataFrame({
        'map_name': unique_maps['map_name'].apply(normalize_map_name),
        'map_id': unique_maps['map_id']
    })
    
    # Remove any duplicate map names (if any) - keep first occurrence
    # Note: If same map_name has different map_ids, we'll keep the first one
    map_name_to_id = map_name_to_id.drop_duplicates(subset='map_name', keep='first')
    
    # Sort by map_id for consistency
    map_name_to_id = map_name_to_id.sort_values('map_id').reset_index(drop=True)
    
    # Save to CSV
    map_name_to_id.to_csv(output_file, index=False)
    
    print(f"\n✅ Created map name to ID mapping with {len(map_name_to_id)} maps")
    print(f"Saved to: {output_file}")
    print("\nMap mappings:")
    print(map_name_to_id)
else:
    print("❌ No map data found in the processed files")

Processing 10 files to extract map information...
Files: ['3dmax.csv', '500.csv', '9-pandas.csv', '9ine.csv', '9z.csv', 'astralis.csv', 'atox.csv', 'aurora.csv', 'b8.csv', 'bcgame.csv']

  ✅ 3dmax.csv: Found 8 unique maps
  ✅ 500.csv: Found 8 unique maps
  ✅ 9-pandas.csv: Found 7 unique maps
  ✅ 9ine.csv: Found 9 unique maps
  ✅ 9z.csv: Found 8 unique maps
  ✅ astralis.csv: Found 8 unique maps
  ✅ atox.csv: Found 7 unique maps
  ✅ aurora.csv: Found 8 unique maps
  ✅ b8.csv: Found 8 unique maps
  ✅ bcgame.csv: Found 8 unique maps

✅ Created map name to ID mapping with 9 maps
Saved to: /Users/jayesh/University/CS-IV-year/COMPSCI 4AL3/Project Progress Report/emils-demos/data/map_name_to_id.csv

Map mappings:
   map_name  map_id
0     dust2      31
1    mirage      32
2   inferno      33
3      nuke      34
4     train      35
5  overpass      40
6   vertigo      46
7   ancient      47
8    anubis      48


In [3]:
# Define paths
project_root = Path().resolve().parent
team_results_dir = project_root / "data" / "team_results"

# Get all CSV files from team_results
team_results_files = sorted(team_results_dir.glob("*.csv"))

print(f"Processing {len(team_results_files)} team files to calculate cumulative wins/losses...\n")

# List to store results for all teams
all_team_map_stats = []

for file_path in team_results_files:
    try:
        # Load team data
        df = pd.read_csv(file_path)
        
        # Ensure we have required columns
        if not all(col in df.columns for col in ['team_name', 'map_name', 'match_date', 'result']):
            print(f"  ⚠️  Skipping {file_path.name}: Missing required columns")
            continue
        
        # Convert match_date to datetime
        df['match_date'] = pd.to_datetime(df['match_date'], errors='coerce')
        
        # Remove rows with invalid dates
        df = df.dropna(subset=['match_date'])
        
        # Sort by date (oldest first) for cumulative calculation
        df = df.sort_values('match_date', ascending=True).reset_index(drop=True)
        
        # Get team name (should be consistent across the file)
        team_name = df['team_name'].iloc[0] if len(df) > 0 else file_path.stem
        
        # Group by map and calculate cumulative wins/losses
        for map_name, map_data in df.groupby('map_name'):
            # Sort by date within this map
            map_data = map_data.sort_values('match_date', ascending=True).reset_index(drop=True)
            
            # Create win/loss indicators
            map_data['is_win'] = (map_data['result'].str.upper().str.strip() == 'W').astype(int)
            map_data['is_loss'] = (map_data['result'].str.upper().str.strip() == 'L').astype(int)
            
            # Calculate cumulative wins and losses
            map_data['cumulative_wins'] = map_data['is_win'].cumsum()
            map_data['cumulative_losses'] = map_data['is_loss'].cumsum()
            
            # Add match number for this team on this map
            map_data['match_number'] = range(1, len(map_data) + 1)
            
            # Select relevant columns
            result_df = map_data[[
                'team_name', 
                'map_name', 
                'match_date', 
                'result',
                'cumulative_wins',
                'cumulative_losses',
                'match_number'
            ]].copy()
            
            all_team_map_stats.append(result_df)
        
        print(f"  ✅ {file_path.name}: Processed {df['map_name'].nunique()} maps, {len(df)} total matches")
        
    except Exception as e:
        print(f"  ❌ Error processing {file_path.name}: {e}")

# Combine all results into a single DataFrame
if all_team_map_stats:
    cumulative_stats_df = pd.concat(all_team_map_stats, ignore_index=True)
    
    # Sort by team_name, map_name, and match_date
    cumulative_stats_df = cumulative_stats_df.sort_values(
        ['team_name', 'map_name', 'match_date'], 
        ascending=[True, True, True]
    ).reset_index(drop=True)
    
    print(f"\n✅ Created cumulative statistics DataFrame")
    print(f"Total rows: {len(cumulative_stats_df)}")
    print(f"Unique teams: {cumulative_stats_df['team_name'].nunique()}")
    print(f"Unique maps: {cumulative_stats_df['map_name'].nunique()}")
    
    # Display sample data
    print("\nSample data (first 20 rows):")
    print(cumulative_stats_df.head(20))
    
    # Display summary statistics
    print("\n\nSummary by team and map (showing first 10 teams):")
    summary = cumulative_stats_df.groupby(['team_name', 'map_name']).agg({
        'cumulative_wins': 'max',
        'cumulative_losses': 'max',
        'match_number': 'max'
    }).rename(columns={
        'cumulative_wins': 'total_wins',
        'cumulative_losses': 'total_losses',
        'match_number': 'total_matches'
    }).reset_index()
    
    # Calculate win rate
    summary['win_rate'] = (summary['total_wins'] / summary['total_matches'] * 100).round(2)
    
    print(summary.head(20))
    
    # Save to CSV (optional)
    output_file = project_root / "data" / "preprocessed_data" / "team_map_cumulative_stats.csv"
    output_file.parent.mkdir(parents=True, exist_ok=True)
    cumulative_stats_df.to_csv(output_file, index=False)
    print(f"\n💾 Saved to: {output_file}")

    
else:
    print("❌ No data processed")

Processing 68 team files to calculate cumulative wins/losses...

  ✅ 3dmax.csv: Processed 8 maps, 199 total matches
  ✅ 500.csv: Processed 8 maps, 306 total matches
  ✅ 9-pandas.csv: Processed 7 maps, 144 total matches
  ✅ 9ine.csv: Processed 9 maps, 414 total matches
  ✅ 9z.csv: Processed 8 maps, 175 total matches
  ✅ astralis.csv: Processed 8 maps, 201 total matches
  ✅ atox.csv: Processed 7 maps, 72 total matches
  ✅ aurora.csv: Processed 8 maps, 300 total matches
  ✅ b8.csv: Processed 8 maps, 207 total matches
  ✅ bcgame.csv: Processed 8 maps, 270 total matches
  ✅ bestia.csv: Processed 7 maps, 201 total matches
  ✅ betboom.csv: Processed 9 maps, 262 total matches
  ✅ betclic.csv: Processed 9 maps, 269 total matches
  ✅ big.csv: Processed 9 maps, 250 total matches
  ✅ bluejays.csv: Processed 8 maps, 176 total matches
  ✅ chinggis-warriors.csv: Processed 8 maps, 219 total matches
  ✅ cloud9.csv: Processed 6 maps, 14 total matches
  ✅ complexity.csv: Processed 8 maps, 97 total matche

In [4]:
import pandas as pd
from pathlib import Path

# Define paths
project_root = Path().resolve().parent
team_results_dir = project_root / "data" / "team_results"

# Normalize team/opponent names (lowercase and replace spaces with hyphens)
def normalize_team_name(name):
    """Normalize team name by lowercasing and replacing spaces with hyphens."""
    if pd.isna(name):
        return name
    return str(name).lower().strip().replace(' ', '-')

# Get all CSV files from team_results
team_results_files = sorted(team_results_dir.glob("*.csv"))

print(f"Processing {len(team_results_files)} team files to calculate cumulative wins/losses against opponents...\n")

# List to store results for all teams
all_team_opponent_stats = []

for file_path in team_results_files:
    try:
        # Load team data
        df = pd.read_csv(file_path)
        
        # Ensure we have required columns
        if not all(col in df.columns for col in ['team_name', 'opponent_name', 'match_date', 'result']):
            print(f"  ⚠️  Skipping {file_path.name}: Missing required columns")
            continue
        
        # Normalize opponent names
        df['opponent_name'] = df['opponent_name'].apply(normalize_team_name)
        
        # Convert match_date to datetime
        df['match_date'] = pd.to_datetime(df['match_date'], errors='coerce')
        
        # Remove rows with invalid dates
        df = df.dropna(subset=['match_date'])
        
        # Sort by date (oldest first) for cumulative calculation
        df = df.sort_values('match_date', ascending=True).reset_index(drop=True)
        
        # Get team name (should be consistent across the file)
        team_name = df['team_name'].iloc[0] if len(df) > 0 else file_path.stem
        team_name = normalize_team_name(team_name)
        df['team_name'] = team_name
        
        # Group by opponent and calculate cumulative wins/losses
        for opponent_name, opponent_data in df.groupby('opponent_name'):
            # Sort by date within this opponent matchup
            opponent_data = opponent_data.sort_values('match_date', ascending=True).reset_index(drop=True)
            
            # Create win/loss indicators
            opponent_data['is_win'] = (opponent_data['result'].str.upper().str.strip() == 'W').astype(int)
            opponent_data['is_loss'] = (opponent_data['result'].str.upper().str.strip() == 'L').astype(int)
            
            # Calculate cumulative wins and losses
            opponent_data['cumulative_wins'] = opponent_data['is_win'].cumsum()
            opponent_data['cumulative_losses'] = opponent_data['is_loss'].cumsum()
            
            # Add match number for this team against this opponent
            opponent_data['match_number'] = range(1, len(opponent_data) + 1)
            
            # Select relevant columns
            result_df = opponent_data[[
                'team_name', 
                'opponent_name',
                'opponent_id',
                'match_date', 
                'result',
                'cumulative_wins',
                'cumulative_losses',
                'match_number'
            ]].copy()
            
            all_team_opponent_stats.append(result_df)
        
        num_opponents = df['opponent_name'].nunique()
        print(f"  ✅ {file_path.name}: Processed {num_opponents} opponents, {len(df)} total matches")
        
    except Exception as e:
        print(f"  ❌ Error processing {file_path.name}: {e}")

# Combine all results into a single DataFrame
if all_team_opponent_stats:
    cumulative_stats_df = pd.concat(all_team_opponent_stats, ignore_index=True)
    
    # Sort by team_name, opponent_name, and match_date
    cumulative_stats_df = cumulative_stats_df.sort_values(
        ['team_name', 'opponent_name', 'match_date'], 
        ascending=[True, True, True]
    ).reset_index(drop=True)
    
    print(f"\n✅ Created cumulative opponent statistics DataFrame")
    print(f"Total rows: {len(cumulative_stats_df)}")
    print(f"Unique teams: {cumulative_stats_df['team_name'].nunique()}")
    print(f"Unique opponents: {cumulative_stats_df['opponent_name'].nunique()}")
    
    # Display sample data
    print("\nSample data (first 20 rows):")
    print(cumulative_stats_df.head(20))
    
    # Display summary statistics
    print("\n\nSummary by team and opponent (showing first 15 matchups):")
    summary = cumulative_stats_df.groupby(['team_name', 'opponent_name']).agg({
        'cumulative_wins': 'max',
        'cumulative_losses': 'max',
        'match_number': 'max',
        'opponent_id': 'first'
    }).rename(columns={
        'cumulative_wins': 'total_wins',
        'cumulative_losses': 'total_losses',
        'match_number': 'total_matches',
        'opponent_id': 'opponent_id'
    }).reset_index()
    
    # Calculate win rate
    summary['win_rate'] = (summary['total_wins'] / summary['total_matches'] * 100).round(2)
    
    print(summary.head(15))
    
    # Save to CSV
    output_file = project_root / "data" / "preprocessed_data" / "team_opponent_cumulative_stats.csv"
    output_file.parent.mkdir(parents=True, exist_ok=True)
    cumulative_stats_df.to_csv(output_file, index=False)
    print(f"\n💾 Saved to: {output_file}")
    
    # Show some interesting statistics
    print("\n\nTop matchups by number of games:")
    matchup_counts = cumulative_stats_df.groupby(['team_name', 'opponent_name']).size().reset_index(name='match_count')
    matchup_counts = matchup_counts.sort_values('match_count', ascending=False)
    print(matchup_counts.head(10))
    
else:
    print("❌ No data processed")

Processing 68 team files to calculate cumulative wins/losses against opponents...

  ✅ 3dmax.csv: Processed 37 opponents, 199 total matches
  ✅ 500.csv: Processed 71 opponents, 306 total matches
  ✅ 9-pandas.csv: Processed 43 opponents, 144 total matches
  ✅ 9ine.csv: Processed 77 opponents, 414 total matches
  ✅ 9z.csv: Processed 39 opponents, 175 total matches
  ✅ astralis.csv: Processed 40 opponents, 201 total matches
  ✅ atox.csv: Processed 19 opponents, 72 total matches
  ✅ aurora.csv: Processed 68 opponents, 300 total matches
  ✅ b8.csv: Processed 56 opponents, 207 total matches
  ✅ bcgame.csv: Processed 60 opponents, 270 total matches
  ✅ bestia.csv: Processed 34 opponents, 201 total matches
  ✅ betboom.csv: Processed 64 opponents, 262 total matches
  ✅ betclic.csv: Processed 66 opponents, 269 total matches
  ✅ big.csv: Processed 67 opponents, 250 total matches
  ✅ bluejays.csv: Processed 34 opponents, 176 total matches
  ✅ chinggis-warriors.csv: Processed 45 opponents, 219 tota

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

# Define paths
project_root = Path().resolve().parent
team_results_dir = project_root / "data" / "team_results"
opponent_stats_file = project_root / "data" / "preprocessed_data" / "team_opponent_cumulative_stats.csv"
map_stats_file = project_root / "data" / "preprocessed_data" / "team_map_cumulative_stats.csv"
rankings_file = project_root / "hltv_team_rankings_original.csv"
team_name_to_id_file = project_root / "data" / "team_name_to_id.csv"
output_file = project_root / "data" / "preprocessed_data" / "match_features.csv"

print("Loading cumulative statistics...")
# Load cumulative statistics
opponent_stats = pd.read_csv(opponent_stats_file)
map_stats = pd.read_csv(map_stats_file)

# Convert dates to datetime
opponent_stats['match_date'] = pd.to_datetime(opponent_stats['match_date'])
map_stats['match_date'] = pd.to_datetime(map_stats['match_date'])

# Normalize team and map names in the stats DataFrames
def normalize_team_name(name):
    """Normalize team name by lowercasing and replacing spaces with hyphens."""
    if pd.isna(name):
        return name
    return str(name).lower().strip().replace(' ', '-')

def normalize_map_name(name):
    """Normalize map name by lowercasing and replacing spaces with hyphens."""
    if pd.isna(name):
        return name
    return str(name).lower().strip().replace(' ', '-')

# Normalize names in the stats DataFrames
opponent_stats['team_name'] = opponent_stats['team_name'].apply(normalize_team_name)
opponent_stats['opponent_name'] = opponent_stats['opponent_name'].apply(normalize_team_name)
map_stats['team_name'] = map_stats['team_name'].apply(normalize_team_name)
map_stats['map_name'] = map_stats['map_name'].apply(normalize_map_name)

print(f"Loaded {len(opponent_stats)} opponent statistics rows")
print(f"Loaded {len(map_stats)} map statistics rows")

print("\nLoading team rankings...")
# Load rankings and team name to ID mapping
rankings_df = pd.read_csv(rankings_file)
team_name_to_id_df = pd.read_csv(team_name_to_id_file)

# Extract rank number from "#1", "#2", etc.
rankings_df['rank'] = rankings_df['rank'].str.replace('#', '').astype(int)

# Normalize team names in rankings
rankings_df['name_normalized'] = rankings_df['name'].apply(normalize_team_name)

# Convert date to datetime
rankings_df['date'] = pd.to_datetime(rankings_df['date'])

# Merge rankings with team IDs
rankings_df = rankings_df.merge(
    team_name_to_id_df,
    left_on='name_normalized',
    right_on='team_name',
    how='left'
)

# Filter out rows where we couldn't find a team ID (teams not in our dataset)
rankings_df = rankings_df.dropna(subset=['team_id'])

print(f"Loaded {len(rankings_df)} ranking entries")
print(f"Unique teams in rankings: {rankings_df['team_id'].nunique()}")
print(f"Date range in rankings: {rankings_df['date'].min()} to {rankings_df['date'].max()}")

# Sort rankings by date and team_id for efficient lookup
rankings_df = rankings_df.sort_values(['date', 'team_id']).reset_index(drop=True)

print("\nLoading all team match data...")
# Load all team result files and combine
all_matches = []

team_results_files = sorted(team_results_dir.glob("*.csv"))
for file_path in team_results_files:
    try:
        df = pd.read_csv(file_path)
        if len(df) > 0:
            # Normalize team and opponent names
            df['team_name'] = df['team_name'].apply(normalize_team_name)
            df['opponent_name'] = df['opponent_name'].apply(normalize_team_name)
            df['match_date'] = pd.to_datetime(df['match_date'], errors='coerce')
            df = df.dropna(subset=['match_date'])
            all_matches.append(df)
    except Exception as e:
        print(f"  ⚠️  Error loading {file_path.name}: {e}")

# Combine all matches
matches_df = pd.concat(all_matches, ignore_index=True)
matches_df = matches_df.sort_values('match_date').reset_index(drop=True)

print(f"Loaded {len(matches_df)} total matches")
print(f"Date range: {matches_df['match_date'].min()} to {matches_df['match_date'].max()}")

print("\nCreating feature dataset...")
# Create feature rows
feature_rows = []

for idx, match in matches_df.iterrows():
    if idx % 1000 == 0:
        print(f"  Processing match {idx+1}/{len(matches_df)}")
    
    team_a = match['team_name']
    team_b = match['opponent_name']
    team_a_id = match['team_id']
    team_b_id = match['opponent_id']
    map_name = normalize_map_name(match['map_name']) if pd.notna(match['map_name']) else match['map_name']
    match_date = match['match_date']
    
    # Get team A's stats against team B BEFORE this match
    team_a_vs_b = opponent_stats[
        (opponent_stats['team_name'] == team_a) & 
        (opponent_stats['opponent_name'] == team_b) &
        (opponent_stats['match_date'] < match_date)
    ]
    
    if len(team_a_vs_b) > 0:
        latest_stats = team_a_vs_b.iloc[-1]
        team_a_wins_vs_b = latest_stats['cumulative_wins']
        team_a_losses_vs_b = latest_stats['cumulative_losses']
        team_a_total_vs_b = team_a_wins_vs_b + team_a_losses_vs_b
        team_a_winrate_vs_b = (team_a_wins_vs_b / team_a_total_vs_b * 100) if team_a_total_vs_b > 0 else 0.0
    else:
        team_a_wins_vs_b = 0
        team_a_losses_vs_b = 0
        team_a_total_vs_b = 0
        team_a_winrate_vs_b = 0.0
    
    # Get team A's stats on this map BEFORE this match
    team_a_map = map_stats[
        (map_stats['team_name'] == team_a) &
        (map_stats['map_name'] == map_name) &
        (map_stats['match_date'] < match_date)
    ]
    
    if len(team_a_map) > 0:
        latest_map_stats_a = team_a_map.iloc[-1]
        team_a_map_wins = latest_map_stats_a['cumulative_wins']
        team_a_map_losses = latest_map_stats_a['cumulative_losses']
        team_a_map_total = team_a_map_wins + team_a_map_losses
        team_a_map_winrate = (team_a_map_wins / team_a_map_total * 100) if team_a_map_total > 0 else 0.0
    else:
        team_a_map_wins = 0
        team_a_map_losses = 0
        team_a_map_total = 0
        team_a_map_winrate = 0.0
    
    # Get team B's stats on this map BEFORE this match
    team_b_map = map_stats[
        (map_stats['team_name'] == team_b) &
        (map_stats['map_name'] == map_name) &
        (map_stats['match_date'] < match_date)
    ]
    
    if len(team_b_map) > 0:
        latest_map_stats_b = team_b_map.iloc[-1]
        team_b_map_wins = latest_map_stats_b['cumulative_wins']
        team_b_map_losses = latest_map_stats_b['cumulative_losses']
        team_b_map_total = team_b_map_wins + team_b_map_losses
        team_b_map_winrate = (team_b_map_wins / team_b_map_total * 100) if team_b_map_total > 0 else 0.0
    else:
        team_b_map_wins = 0
        team_b_map_losses = 0
        team_b_map_total = 0
        team_b_map_winrate = 0.0
    
    # Get team A's global ranking at or before this match date
    team_a_rankings = rankings_df[
        (rankings_df['team_id'] == team_a_id) &
        (rankings_df['date'] <= match_date)
    ]
    
    if len(team_a_rankings) > 0:
        # Get the most recent ranking
        latest_ranking_a = team_a_rankings.iloc[-1]
        team_a_global_ranking = latest_ranking_a['points']
    else:
        # No ranking found - use a low number (e.g., 0) to indicate unranked
        team_a_global_ranking = 0
    
    # Get team B's global ranking at or before this match date
    team_b_rankings = rankings_df[
        (rankings_df['team_id'] == team_b_id) &
        (rankings_df['date'] <= match_date)
    ]
    
    if len(team_b_rankings) > 0:
        # Get the most recent ranking
        latest_ranking_b = team_b_rankings.iloc[-1]
        team_b_global_ranking = latest_ranking_b['points']
    else:
        # No ranking found - use a low number (e.g., 0) to indicate unranked
        team_b_global_ranking = 0
    
    # Determine if team A won
    result = match['result'].upper().strip()
    team_a_won = 1 if result == 'W' else 0
    
    # Create feature row
    feature_row = {
        'team_a_id': team_a_id,
        'team_b_id': team_b_id,
        'map_id': match['map_id'],
        'match_date': match_date,
        # Team A vs Team B stats (up to this match)
        'team_a_wins_vs_b': team_a_wins_vs_b,
        'team_a_losses_vs_b': team_a_losses_vs_b,
        'team_a_total_vs_b': team_a_total_vs_b,
        'team_a_winrate_vs_b': round(team_a_winrate_vs_b, 2),
        # Team A map stats (up to this match)
        'team_a_map_wins': team_a_map_wins,
        'team_a_map_losses': team_a_map_losses,
        'team_a_map_total': team_a_map_total,
        'team_a_map_winrate': round(team_a_map_winrate, 2),
        # Team B map stats (up to this match)
        'team_b_map_wins': team_b_map_wins,
        'team_b_map_losses': team_b_map_losses,
        'team_b_map_total': team_b_map_total,
        'team_b_map_winrate': round(team_b_map_winrate, 2),
        # Global rankings
        'team_a_global_ranking': team_a_global_ranking,
        'team_b_global_ranking': team_b_global_ranking,
        # Target variable
        'team_a_won': team_a_won
    }
    
    feature_rows.append(feature_row)

# Create feature DataFrame
features_df = pd.DataFrame(feature_rows)

print(f"\n✅ Created feature dataset with {len(features_df)} matches")
print(f"\nFeature columns: {list(features_df.columns)}")

# Display statistics
print("\nDataset Statistics:")
print(f"  Total matches: {len(features_df)}")
print(f"  Team A wins: {features_df['team_a_won'].sum()} ({features_df['team_a_won'].mean()*100:.1f}%)")
print(f"  Team A losses: {(1-features_df['team_a_won']).sum()} ({(1-features_df['team_a_won']).mean()*100:.1f}%)")
print(f"  Unique teams: {pd.concat([features_df['team_a_id'], features_df['team_b_id']]).nunique()}")
print(f"  Unique maps: {features_df['map_id'].nunique()}")

# Show sample of data
print("\nSample features (first 10 rows):")
print(features_df.head(10))

# Check for matches with rankings
has_ranking_a = features_df['team_a_global_ranking'] != 0
has_ranking_b = features_df['team_b_global_ranking'] != 0
has_any_ranking = has_ranking_a | has_ranking_b

print(f"\nMatches with team A ranking: {has_ranking_a.sum()} ({has_ranking_a.mean()*100:.1f}%)")
print(f"Matches with team B ranking: {has_ranking_b.sum()} ({has_ranking_b.mean()*100:.1f}%)")
print(f"Matches with any ranking: {has_any_ranking.sum()} ({has_any_ranking.mean()*100:.1f}%)")

# Check for matches with map stats
has_map_stats_a = features_df['team_a_map_total'] > 0
has_map_stats_b = features_df['team_b_map_total'] > 0
has_any_map_stats = has_map_stats_a | has_map_stats_b

print(f"Matches with team A map stats: {has_map_stats_a.sum()} ({has_map_stats_a.mean()*100:.1f}%)")
print(f"Matches with team B map stats: {has_map_stats_b.sum()} ({has_map_stats_b.mean()*100:.1f}%)")
print(f"Matches with any map stats: {has_any_map_stats.sum()} ({has_any_map_stats.mean()*100:.1f}%)")

# Save to CSV
features_df.to_csv(output_file, index=False)
print(f"\n💾 Saved feature dataset to: {output_file}")
print(f"   Shape: {features_df.shape}")

# Display feature summary
print("\n\nFeature Summary:")
print(features_df[['team_a_wins_vs_b', 'team_a_losses_vs_b', 'team_a_winrate_vs_b', 
                   'team_a_map_winrate', 'team_b_map_winrate', 
                   'team_a_global_ranking', 'team_b_global_ranking', 'team_a_won']].describe())

Loading cumulative statistics...
Loaded 14428 opponent statistics rows
Loaded 14428 map statistics rows

Loading team rankings...
Loaded 3307 ranking entries
Unique teams in rankings: 68
Date range in rankings: 2024-11-04 00:00:00 to 2025-11-03 00:00:00

Loading all team match data...
Loaded 14428 total matches
Date range: 2024-11-04 00:00:00 to 2025-12-29 00:00:00

Creating feature dataset...
  Processing match 1/14428
  Processing match 1001/14428
  Processing match 2001/14428
  Processing match 3001/14428
  Processing match 4001/14428
  Processing match 5001/14428
  Processing match 6001/14428
  Processing match 7001/14428
  Processing match 8001/14428
  Processing match 9001/14428
  Processing match 10001/14428
  Processing match 11001/14428
  Processing match 12001/14428
  Processing match 13001/14428
  Processing match 14001/14428

✅ Created feature dataset with 14428 matches

Feature columns: ['team_a_id', 'team_b_id', 'map_id', 'match_date', 'team_a_wins_vs_b', 'team_a_losses_v

In [6]:
import pandas as pd
from pathlib import Path

# Define paths
project_root = Path().resolve().parent
input_file = project_root / "data" / "preprocessed_data" / "match_features.csv"
output_file = project_root / "data" / "preprocessed_data" / "final_features.csv"

print("Loading match features...")
# Load match features
df = pd.read_csv(input_file)

print(f"Loaded {len(df)} rows")
print(f"Original columns: {list(df.columns)}")
print(f"\nUnique map_ids: {sorted(df['map_id'].unique())}")

# Remove team_a_id, team_b_id, and match_date
columns_to_remove = ['team_a_id', 'team_b_id', 'match_date']
df_cleaned = df.drop(columns=columns_to_remove)

print(f"\nRemoved columns: {columns_to_remove}")

# One-hot encode map_id
print("\nOne-hot encoding map_id...")
# Get dummies for map_id - convert to int to get 0/1 instead of True/False
map_dummies = pd.get_dummies(df_cleaned['map_id'], prefix='map_id', dtype=int)

# Remove original map_id column
df_without_map = df_cleaned.drop(columns=['map_id'])

# Combine: map columns first (left side), then other columns
df_final = pd.concat([map_dummies, df_without_map], axis=1)

print(f"Created {len(map_dummies.columns)} one-hot encoded map columns")
print(f"Map columns: {list(map_dummies.columns)}")
print(f"\nMap columns are now on the LEFT side of the dataframe")

# Display sample
print("\nSample of final features (first 5 rows):")
print(df_final.head())

# Verify one-hot encoding values are 0/1
print(f"\nOne-hot encoding values: {sorted(df_final[map_dummies.columns].values.flatten())}")
print(f"Unique values in map columns: {set(df_final[map_dummies.columns].values.flatten())}")

# Display summary statistics
print("\n\nFeature Summary:")
print(df_final.describe())

# Save to CSV
df_final.to_csv(output_file, index=False)
print(f"\n💾 Saved final features to: {output_file}")
print(f"   Shape: {df_final.shape}")
print(f"   Target variable: team_a_won")
print(f"   Target distribution: {df_final['team_a_won'].value_counts().to_dict()}")
print(f"\n   Column order: {list(df_final.columns[:5])} ... {list(df_final.columns[-3:])}")

Loading match features...
Loaded 14428 rows
Original columns: ['team_a_id', 'team_b_id', 'map_id', 'match_date', 'team_a_wins_vs_b', 'team_a_losses_vs_b', 'team_a_total_vs_b', 'team_a_winrate_vs_b', 'team_a_map_wins', 'team_a_map_losses', 'team_a_map_total', 'team_a_map_winrate', 'team_b_map_wins', 'team_b_map_losses', 'team_b_map_total', 'team_b_map_winrate', 'team_a_global_ranking', 'team_b_global_ranking', 'team_a_won']

Unique map_ids: [np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(40), np.int64(46), np.int64(47), np.int64(48)]

Removed columns: ['team_a_id', 'team_b_id', 'match_date']

One-hot encoding map_id...
Created 9 one-hot encoded map columns
Map columns: ['map_id_31', 'map_id_32', 'map_id_33', 'map_id_34', 'map_id_35', 'map_id_40', 'map_id_46', 'map_id_47', 'map_id_48']

Map columns are now on the LEFT side of the dataframe

Sample of final features (first 5 rows):
   map_id_31  map_id_32  map_id_33  map_id_34  map_id_35  map_id_40  \
0    